In [1]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-v0_8-notebook')

Set up the environment using information given

In [3]:
FUEL_TANK_CAPACITY = 12000
MAX_FUEL_PURCHASE = 10000
FUEL_SAFETY_MARGIN = 600
CITIES = ['Los Angeles', 'Houston', 'New York', 'Miami']
outbound_segment_distance = np.array([1500, 1700, 1300, 2700])
local_fuel_price = np.array([0.88, 0.15, 1.05, 0.95])


Set up output variables with initial values

In [ ]:
fuel_quantity_purchased = np.zeros(len(CITIES))
fuel_at_landing = np.full(len(CITIES), FUEL_SAFETY_MARGIN)
fuel_at_takeoff = np.full(len(CITIES), FUEL_SAFETY_MARGIN)

Define helper functions

In [12]:
# Cell 3: Helper Functions for Circular Route and Fuel Calculations

def get_previous_city_index(current_index):
   """
   Returns the index of the previous city in the circular route.
   For Los Angeles (index 0), returns index of Miami (last city)
   """
   return (current_index - 1) % len(CITIES)

def calculate_fuel_at_landing(current_city_index, fuel_at_takeoff, outbound_distances):
    """
    Calculate fuel at landing for current city based on:
    (1 + (d/4000))*y = (1 - (d/4000))*z - d
    where:
    y = fuel at landing (what we're solving for)
    z = fuel at takeoff from previous city
    d = distance from previous city
    """
    prev_idx = get_previous_city_index(current_city_index)
    prev_takeoff = fuel_at_takeoff[prev_idx]
    prev_distance = outbound_distances[prev_idx]
    
    fuel_at_landing = ((1 - (prev_distance/4000)) * prev_takeoff - prev_distance) / (1 + (prev_distance/4000))
    return fuel_at_landing

# Test the functions
print("Previous city index test:")
for i, city in enumerate(CITIES):
    prev_idx = get_previous_city_index(i)
    print(f"Current city: {city:12} (index {i}) -> Previous city: {CITIES[prev_idx]:12} (index {prev_idx})")

print("\nFuel at landing calculation test:")
test_takeoff = np.full(len(CITIES), 5000)  # example takeoff fuel for all cities
for i, city in enumerate(CITIES):
    prev_idx = get_previous_city_index(i)
    landing_fuel = calculate_fuel_at_landing(i, test_takeoff, outbound_segment_distance)
    print(f"\nRoute: {CITIES[prev_idx]:10} -> {city:12}")
    print(f"Distance: {outbound_segment_distance[prev_idx]} miles")
    print(f"Takeoff fuel: {test_takeoff[prev_idx]} gallons")
    print(f"Landing fuel: {landing_fuel:.2f} gallons")

Previous city index test:
Current city: Los Angeles  (index 0) -> Previous city: Miami        (index 3)
Current city: Houston      (index 1) -> Previous city: Los Angeles  (index 0)
Current city: New York     (index 2) -> Previous city: Houston      (index 1)
Current city: Miami        (index 3) -> Previous city: New York     (index 2)

Fuel at landing calculation test:

Route: Miami      -> Los Angeles 
Distance: 2700 miles
Takeoff fuel: 5000 gallons
Landing fuel: -641.79 gallons

Route: Los Angeles -> Houston     
Distance: 1500 miles
Takeoff fuel: 5000 gallons
Landing fuel: 1181.82 gallons

Route: Houston    -> New York    
Distance: 1700 miles
Takeoff fuel: 5000 gallons
Landing fuel: 824.56 gallons

Route: New York   -> Miami       
Distance: 1300 miles
Takeoff fuel: 5000 gallons
Landing fuel: 1566.04 gallons


In [13]:
# Constraint Functions

def constraint_fuel_safety_margin(fuel_at_landing):
    """
    Ensure landing fuel is above safety margin at all cities
    Must be >= 0 to satisfy constraint
    """
    return fuel_at_landing - FUEL_SAFETY_MARGIN

def constraint_max_fuel_purchased(fuel_quantity_purchased):
    """
    Ensure purchased fuel is below maximum at all cities
    Must be >= 0 to satisfy constraint
    """
    return MAX_FUEL_PURCHASE - fuel_quantity_purchased

def constraint_fuel_tank_capacity(fuel_at_takeoff):
    """
    Ensure fuel at takeoff is below tank capacity at all cities
    Must be >= 0 to satisfy constraint
    """
    return FUEL_TANK_CAPACITY - fuel_at_takeoff

def constraint_fuel_balance(fuel_at_takeoff, fuel_at_landing, fuel_quantity_purchased):
    """
    Ensure fuel balance equation is satisfied at all cities
    Takeoff fuel = Landing fuel + Purchased fuel
    Must = 0 to satisfy constraint
    """
    return fuel_at_takeoff - (fuel_at_landing + fuel_quantity_purchased)

# Test the constraints with some example values
test_landing = np.array([1000, 800, 900, 1200])
test_purchased = np.array([5000, 4000, 6000, 3000])
test_takeoff = test_landing + test_purchased  # This should satisfy the fuel balance

print("Test Constraints:")
print("\nSafety Margin Constraint (should be >= 0):")
print(constraint_fuel_safety_margin(test_landing))

print("\nMax Fuel Purchase Constraint (should be >= 0):")
print(constraint_max_fuel_purchased(test_purchased))

print("\nFuel Tank Capacity Constraint (should be >= 0):")
print(constraint_fuel_tank_capacity(test_takeoff))

print("\nFuel Balance Constraint (should = 0):")
print(constraint_fuel_balance(test_takeoff, test_landing, test_purchased))

Test Constraints:

Safety Margin Constraint (should be >= 0):
[400 200 300 600]

Max Fuel Purchase Constraint (should be >= 0):
[5000 6000 4000 7000]

Fuel Tank Capacity Constraint (should be >= 0):
[6000 7200 5100 7800]

Fuel Balance Constraint (should = 0):
[0 0 0 0]


In [14]:
# Objective Function

def objective_function(x):
   """
   Calculate total fuel cost across all cities.
   x is a 1D array of fuel quantities purchased at each city.
   
   Returns: Total cost to be minimized
   """
   # Reshape input if necessary (scipy.optimize sometimes flattens the input)
   fuel_quantity_purchased = x.reshape(len(CITIES))
   
   # Calculate total cost: sum of (quantity * price) for each city
   total_cost = np.sum(fuel_quantity_purchased * local_fuel_price)
   
   return total_cost

# Test the objective function
test_purchases = np.array([5000, 4000, 6000, 3000])
test_cost = objective_function(test_purchases)

print("Test Objective Function:")
print("\nFuel Purchases:")
for city, quantity, price in zip(CITIES, test_purchases, local_fuel_price):
   print(f"{city:12}: {quantity:5.0f} gallons at ${price:.2f}/gallon = ${quantity*price:.2f}")
print(f"\nTotal Cost: ${test_cost:.2f}")

Test Objective Function:

Fuel Purchases:
Los Angeles :  5000 gallons at $0.88/gallon = $4400.00
Houston     :  4000 gallons at $0.15/gallon = $600.00
New York    :  6000 gallons at $1.05/gallon = $6300.00
Miami       :  3000 gallons at $0.95/gallon = $2850.00

Total Cost: $14150.00


In [16]:
# Cell 7: Set up and Run Optimization

def calculate_fuel_levels(fuel_quantity_purchased):
    """
    Calculate takeoff and landing fuel levels for all cities based on purchased quantities
    """
    fuel_at_takeoff = np.zeros(len(CITIES))
    fuel_at_landing = np.zeros(len(CITIES))
    
    for i in range(len(CITIES)):
        fuel_at_landing[i] = calculate_fuel_at_landing(i, fuel_at_takeoff, outbound_segment_distance)
        fuel_at_takeoff[i] = fuel_at_landing[i] + fuel_quantity_purchased[i]
        
    return fuel_at_takeoff, fuel_at_landing

def prepare_constraints():
    """
    Prepare constraints for scipy.optimize.minimize
    """
    constraints = []
    
    # Fuel balance constraint (equality)
    def fuel_balance_all_cities(x):
        fuel_quantity_purchased = x.reshape(len(CITIES))
        fuel_at_takeoff, fuel_at_landing = calculate_fuel_levels(fuel_quantity_purchased)
        return constraint_fuel_balance(fuel_at_takeoff, fuel_at_landing, fuel_quantity_purchased)
    
    # Safety margin constraint (inequality)
    def safety_margin_all_cities(x):
        fuel_quantity_purchased = x.reshape(len(CITIES))
        fuel_at_takeoff, fuel_at_landing = calculate_fuel_levels(fuel_quantity_purchased)
        return constraint_fuel_safety_margin(fuel_at_landing)
    
    # Tank capacity constraint (inequality)
    def tank_capacity_all_cities(x):
        fuel_quantity_purchased = x.reshape(len(CITIES))
        fuel_at_takeoff, fuel_at_landing = calculate_fuel_levels(fuel_quantity_purchased)
        return constraint_fuel_tank_capacity(fuel_at_takeoff)
    
    constraints.extend([
        {'type': 'eq', 'fun': fuel_balance_all_cities},
        {'type': 'ineq', 'fun': safety_margin_all_cities},
        {'type': 'ineq', 'fun': tank_capacity_all_cities}
    ])
    
    return constraints

# Test initial guess feasibility
x0 = np.full(len(CITIES), MAX_FUEL_PURCHASE/2)
initial_takeoff, initial_landing = calculate_fuel_levels(x0)

print("Initial Guess Diagnostics:")
print("\nFuel Balance Check:")
for i, city in enumerate(CITIES):
    balance = initial_takeoff[i] - (initial_landing[i] + x0[i])
    print(f"{city:12}: Balance error = {balance:.2f}")

print("\nSafety Margin Check:")
for i, city in enumerate(CITIES):
    margin = initial_landing[i] - FUEL_SAFETY_MARGIN
    print(f"{city:12}: Safety margin = {margin:.2f}")

print("\nTank Capacity Check:")
for i, city in enumerate(CITIES):
    capacity = FUEL_TANK_CAPACITY - initial_takeoff[i]
    print(f"{city:12}: Remaining capacity = {capacity:.2f}")

# Try different initial guesses
initial_guesses = [
    np.full(len(CITIES), 1000),  # smaller constant
    np.array([2000, 3000, 2000, 3000]),  # varied values
    np.array([1000, 1000, 1000, 1000])   # minimal values
]

for i, guess in enumerate(initial_guesses):
    print(f"\nTrying optimization with initial guess {i+1}")
    result = minimize(
        objective_function,
        guess,
        method='SLSQP',
        bounds=bounds,
        constraints=prepare_constraints(),
        options={'ftol': 1e-6, 'maxiter': 1000}
    )
    print(f"Success: {result.success}")
    print(f"Message: {result.message}")
    if result.success:
        print("Solution found!")
        break

Initial Guess Diagnostics:

Fuel Balance Check:
Los Angeles : Balance error = 0.00
Houston     : Balance error = 0.00
New York    : Balance error = 0.00
Miami       : Balance error = 0.00

Safety Margin Check:
Los Angeles : Safety margin = -2211.94
Houston     : Safety margin = -150.88
New York    : Safety margin = 405.78
Miami       : Safety margin = 1478.42

Tank Capacity Check:
Los Angeles : Remaining capacity = 8611.94
Houston     : Remaining capacity = 6550.88
New York    : Remaining capacity = 5994.22
Miami       : Remaining capacity = 4921.58

Trying optimization with initial guess 1
Success: False
Message: Singular matrix C in LSQ subproblem

Trying optimization with initial guess 2
Success: False
Message: Singular matrix C in LSQ subproblem

Trying optimization with initial guess 3
Success: False
Message: Singular matrix C in LSQ subproblem
